# Reading in the files and imports

In [1]:
import pyspark.sql.functions as F
import pandas as pd 
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np 

import os 
import pickle
import re
from datetime import datetime
import requests
import pytz
import pandas as pd
import numpy as np
import ast


In [2]:

import pyspark
# create spark context
sc = pyspark.SparkContext()
# create spark session 
spark = pyspark.sql.SparkSession(sc)

path_json = ".././../data/Topic_vegan/*.json" 

df_json = spark.read.option("multiline","true").json(path_json)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/25 15:52:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


22/11/25 15:53:29 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
spark

## Selecting the correct columns, converting types and checking for doubles 

In [3]:
# twitter represents likes as hearts 
# the number of likes different topics receive over time 
# use favorites_count 

#plot aantal tweets over een topic over time 

df_json_sub = df_json.select(F.col("user.name"),
                                F.col("user.screen_name"),
                                F.col("created_at"), 
                                F.col("full_text"),
                                F.col("entities.hashtags"),
                                F.col("lang"),
                                F.col("favorite_count"),
                                F.col("user.followers_count"),
                                F.col("user.friends_count"),
                                F.col("user.favourites_count"),
                                F.col("entities.urls"),
                                F.col("entities.symbols"), 
                                )
#date --> time stamp variable 
def getDate(date):
    if date is not None:
        return str(datetime.strptime(date,'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC).strftime("%Y-%m-%d %H:%M:%S"))
    else:
        return None
date_udf = F.udf(getDate, StringType())
df_json_sub = df_json_sub.withColumn('post_created_at', F.to_utc_timestamp(date_udf("created_at"), "UTC"))

#extract year, month, day 

df_json_sub = df_json_sub.withColumn('year', year('post_created_at'))
df_json_sub = df_json_sub.withColumn('month', month('post_created_at'))
df_json_sub = df_json_sub.withColumn('day', dayofmonth('post_created_at')) #number from 1 to 31 

df_json_sub.count()



3428559

In [4]:
#drop duplicates and retweets 
df_json_sub = df_json_sub.filter(~F.col("full_text").startswith("RT"))\
                        .drop_duplicates().cache()
#sorting such when dropping later we only keep the most recent post 
df_json_sub = df_json_sub.sort("post_created_at", ascending=False)
#removing spam accounts 
df_json_sub = df_json_sub.drop_duplicates(["full_text", "screen_name"])
                        
df_json_sub.printSchema()
df_json_sub.count() #1340938 

root
 |-- name: string (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- full_text: string (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- indices: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- text: string (nullable = true)
 |-- lang: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- favourites_count: long (nullable = true)
 |-- urls: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- display_url: string (nullable = true)
 |    |    |-- expanded_url: string (nullable = true)
 |    |    |-- indices: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- url: string (nullable = true)
 |-- symbols: array (nullable = true)
 |   

22/11/25 15:54:47 WARN MemoryStore: Not enough space to cache rdd_23_95 in memory! (computed 3.8 MiB so far)
22/11/25 15:54:47 WARN BlockManager: Persisting block rdd_23_95 to disk instead.


22/11/25 15:54:47 WARN MemoryStore: Not enough space to cache rdd_23_95 in memory! (computed 3.8 MiB so far)
22/11/25 15:54:47 WARN MemoryStore: Not enough space to cache rdd_23_97 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:47 WARN BlockManager: Persisting block rdd_23_97 to disk instead.
22/11/25 15:54:47 WARN MemoryStore: Not enough space to cache rdd_23_97 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:47 WARN MemoryStore: Not enough space to cache rdd_23_98 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:47 WARN BlockManager: Persisting block rdd_23_98 to disk instead.


22/11/25 15:54:47 WARN MemoryStore: Not enough space to cache rdd_23_99 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:47 WARN BlockManager: Persisting block rdd_23_99 to disk instead.
22/11/25 15:54:47 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_23_104 in memory.
22/11/25 15:54:47 WARN MemoryStore: Not enough space to cache rdd_23_101 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:47 WARN MemoryStore: Not enough space to cache rdd_23_104 in memory! (computed 384.0 B so far)
22/11/25 15:54:47 WARN BlockManager: Persisting block rdd_23_104 to disk instead.
22/11/25 15:54:47 WARN BlockManager: Persisting block rdd_23_101 to disk instead.
22/11/25 15:54:47 WARN MemoryStore: Not enough space to cache rdd_23_100 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:47 WARN BlockManager: Persisting block rdd_23_100 to disk instead.
22/11/25 15:54:47 WARN MemoryStore: Not enough space to cache rdd_23_101 in memory! (computed 3.9 M

22/11/25 15:54:47 WARN MemoryStore: Not enough space to cache rdd_23_106 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:47 WARN BlockManager: Persisting block rdd_23_106 to disk instead.
22/11/25 15:54:47 WARN MemoryStore: Not enough space to cache rdd_23_104 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:47 WARN MemoryStore: Not enough space to cache rdd_23_106 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:48 WARN MemoryStore: Not enough space to cache rdd_23_107 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:48 WARN BlockManager: Persisting block rdd_23_107 to disk instead.
22/11/25 15:54:48 WARN MemoryStore: Not enough space to cache rdd_23_108 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:48 WARN BlockManager: Persisting block rdd_23_108 to disk instead.


...

22/11/25 15:54:51 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_23_172 in memory.
22/11/25 15:54:51 WARN MemoryStore: Not enough space to cache rdd_23_172 in memory! (computed 384.0 B so far)
22/11/25 15:54:51 WARN BlockManager: Persisting block rdd_23_172 to disk instead.
22/11/25 15:54:51 WARN MemoryStore: Not enough space to cache rdd_23_168 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:51 WARN BlockManager: Persisting block rdd_23_168 to disk instead.


22/11/25 15:54:51 WARN MemoryStore: Not enough space to cache rdd_23_175 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:51 WARN BlockManager: Persisting block rdd_23_175 to disk instead.
22/11/25 15:54:51 WARN MemoryStore: Not enough space to cache rdd_23_175 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:52 WARN MemoryStore: Not enough space to cache rdd_23_176 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:52 WARN BlockManager: Persisting block rdd_23_176 to disk instead.
22/11/25 15:54:52 WARN MemoryStore: Not enough space to cache rdd_23_180 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:52 WARN BlockManager: Persisting block rdd_23_180 to disk instead.
22/11/25 15:54:52 WARN MemoryStore: Not enough space to cache rdd_23_180 in memory! (computed 3.9 MiB so far)


22/11/25 15:54:52 WARN MemoryStore: Not enough space to cache rdd_23_188 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:52 WARN BlockManager: Persisting block rdd_23_188 to disk instead.
22/11/25 15:54:52 WARN MemoryStore: Not enough space to cache rdd_23_191 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:52 WARN BlockManager: Persisting block rdd_23_191 to disk instead.
22/11/25 15:54:52 WARN MemoryStore: Not enough space to cache rdd_23_189 in memory! (computed 3.9 MiB so far)
22/11/25 15:54:52 WARN BlockManager: Persisting block rdd_23_189 to disk instead.


22/11/25 15:54:52 WARN MemoryStore: Not enough space to cache rdd_23_194 in memory! (computed 3.8 MiB so far)
22/11/25 15:54:52 WARN BlockManager: Persisting block rdd_23_194 to disk instead.
22/11/25 15:54:52 WARN MemoryStore: Not enough space to cache rdd_23_194 in memory! (computed 3.8 MiB so far)


1340938

# Feature Engineering 

In [7]:
# define function to count hashtags
def get_hashtags(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "#" in word:
            counter += 1
    return(counter) 
# define function to count mentions
def get_mentions(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "@" in word:
            counter += 1
    return(counter)
# define function to count exclamation marks
def get_exclamation_marks(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "!" in word:
            counter += 1
    return(counter)
# define function to count number of emojis used
import emojis
def emoji_counter(text):
    nr_emojis = emojis.count(text)
    return(nr_emojis)
# register functions as udf
get_hashtags_UDF = F.udf(get_hashtags, IntegerType())
get_mentions_UDF = F.udf(get_mentions, IntegerType())
get_exclamation_marks_UDF = F.udf(get_exclamation_marks, IntegerType())
emoji_counter_udf = F.udf(emoji_counter, IntegerType())
